In [1]:
from __future__ import print_function

import sys, os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import tensorflow as tf
import tensorflow.keras
import pandas as pd
import numpy as np
import sklearn
import keras
import csv

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout

from keras_uncertainty.models import MCDropoutClassifier, MCDropoutRegressor
from keras_uncertainty.utils import numpy_regression_nll


/home/dfki.uni-bremen.de/bmanickavasakan/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/dfki.uni-bremen.de/bmanickavasakan/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/dfki.uni-bremen.de/bmanickavasakan/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/d

In [2]:
def load_joint_space_csv_chunks(file_path):
    data_frame = pd.read_csv(file_path, skiprows=1, header=None)
    del data_frame[18]
    return data_frame

def load_task_space_csv_chunks(file_path):
    return pd.read_csv(file_path, skiprows=1, header=None)


In [3]:
##please select the appropriate folder, willl use os.path.join() for completed script
TRAIN_FOLDER = '/home/dfki.uni-bremen.de/bmanickavasakan/newdataset_rh5_leg/leg_5steps/'
TEST_FOLDER = '/home/dfki.uni-bremen.de/bmanickavasakan/newdataset_rh5_leg/leg_5steps/test_4steps'

X_TRAIN_FILE = os.path.join(TRAIN_FOLDER, 'leg_forwardkinematics_x.csv')
Q_TRAIN_FILE = os.path.join(TRAIN_FOLDER, 'leg_sysstate_q.csv')
x_train = load_task_space_csv_chunks(X_TRAIN_FILE)
q_train = load_joint_space_csv_chunks(Q_TRAIN_FILE)

X_TEST_FILE = os.path.join(TEST_FOLDER, 'leg_forwardkinematics_x.csv')
Q_TEST_FILE = os.path.join(TEST_FOLDER, 'leg_sysstate_q.csv')
x_test = load_task_space_csv_chunks(X_TEST_FILE)
q_test = load_joint_space_csv_chunks(Q_TEST_FILE)

In [4]:

x_scaler = MinMaxScaler()
q_scaler = MinMaxScaler()

#complete test set
x_train = x_scaler.fit_transform(x_train)
q_train = q_scaler.fit_transform(q_train)
x_test = x_scaler.transform(x_test)
q_test = q_scaler.transform(q_test)


In [5]:
def model_builder(input_shape, output_shape):
    def build_model(depth, width, reduction_factor):
        model = Sequential()

        for i in range(depth):
            num_neurons = max(int(width * (reduction_factor ** i)), 4)
            if i == 0:
                model.add(Dense(num_neurons, activation='relu', input_shape=(input_shape,)))
            else:
                model.add(Dense(num_neurons, activation='relu'))
                model.add(Dropout(0.5))

            model.add(BatchNormalization())

        model.add(Dense(output_shape, activation='sigmoid'))

        model.compile(loss='mse', optimizer='adam', metrics=["mae"])

        return model
    return build_model


In [6]:
HYPERPARAMETERS = {'depth': 6, 'width': 64, 'reduction_factor':  1.1}
with tf.device('/cpu:0'):
    model = model_builder(9, 18)(**HYPERPARAMETERS)
    hist = model.fit(x_train, q_train, epochs = 200, batch_size = 128, verbose = 1, validation_data=(x_test, q_test))


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Train on 15625 samples, validate on 4096 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/200
15625/15625 [==============================] - 2s 103us/sample - loss: 0.1306 - mean_absolute_error: 0.2946 - val_loss: 0.1094 - val_mean_absolute_error: 0.2792
Epoch 2/200
15625/15625 [==============================] - 1s 47us/sample - loss: 0.0940 - mean_absolute_error: 0.2481 - val_loss: 0.1027 - val_mean_absolute_error: 0.2687
Epoch 3/200
15625/15625 [==============================] - 1s 47us/sample - loss: 0.0800 - mean_absolute_error: 0.2275 - val_loss: 0.0747 - val_mean_absolute_error: 0.2261
Epoch 4/200
15625/15625 [==============================] - 1s 46us/sample - loss: 0.0714 - mean_absolute_error: 0.2135 - val_loss: 0.0543 - val_mean_ab

15625/15625 [==============================] - 1s 48us/sample - loss: 0.0389 - mean_absolute_error: 0.1469 - val_loss: 0.0205 - val_mean_absolute_error: 0.0971
Epoch 41/200
15625/15625 [==============================] - 1s 48us/sample - loss: 0.0388 - mean_absolute_error: 0.1469 - val_loss: 0.0206 - val_mean_absolute_error: 0.0971
Epoch 42/200
15625/15625 [==============================] - 1s 49us/sample - loss: 0.0401 - mean_absolute_error: 0.1493 - val_loss: 0.0208 - val_mean_absolute_error: 0.0981
Epoch 43/200
15625/15625 [==============================] - 1s 51us/sample - loss: 0.0387 - mean_absolute_error: 0.1465 - val_loss: 0.0204 - val_mean_absolute_error: 0.0960
Epoch 44/200
15625/15625 [==============================] - 1s 46us/sample - loss: 0.0382 - mean_absolute_error: 0.1452 - val_loss: 0.0206 - val_mean_absolute_error: 0.0968
Epoch 45/200
15625/15625 [==============================] - 1s 50us/sample - loss: 0.0378 - mean_absolute_error: 0.1442 - val_loss: 0.0202 - val_mea

15625/15625 [==============================] - 1s 47us/sample - loss: 0.0314 - mean_absolute_error: 0.1262 - val_loss: 0.0196 - val_mean_absolute_error: 0.0884
Epoch 88/200
15625/15625 [==============================] - 1s 47us/sample - loss: 0.0312 - mean_absolute_error: 0.1257 - val_loss: 0.0186 - val_mean_absolute_error: 0.0851
Epoch 89/200
15625/15625 [==============================] - 1s 47us/sample - loss: 0.0316 - mean_absolute_error: 0.1268 - val_loss: 0.0191 - val_mean_absolute_error: 0.0863
Epoch 90/200
15625/15625 [==============================] - 1s 47us/sample - loss: 0.0313 - mean_absolute_error: 0.1262 - val_loss: 0.0185 - val_mean_absolute_error: 0.0861
Epoch 91/200
15625/15625 [==============================] - 1s 46us/sample - loss: 0.0309 - mean_absolute_error: 0.1249 - val_loss: 0.0197 - val_mean_absolute_error: 0.0887
Epoch 92/200
15625/15625 [==============================] - 1s 46us/sample - loss: 0.0310 - mean_absolute_error: 0.1251 - val_loss: 0.0188 - val_mea

15625/15625 [==============================] - 1s 47us/sample - loss: 0.0304 - mean_absolute_error: 0.1226 - val_loss: 0.0186 - val_mean_absolute_error: 0.0845
Epoch 135/200
15625/15625 [==============================] - 1s 47us/sample - loss: 0.0301 - mean_absolute_error: 0.1219 - val_loss: 0.0189 - val_mean_absolute_error: 0.0874
Epoch 136/200
15625/15625 [==============================] - 1s 49us/sample - loss: 0.0300 - mean_absolute_error: 0.1213 - val_loss: 0.0185 - val_mean_absolute_error: 0.0842
Epoch 137/200
15625/15625 [==============================] - 1s 46us/sample - loss: 0.0300 - mean_absolute_error: 0.1212 - val_loss: 0.0188 - val_mean_absolute_error: 0.0856
Epoch 138/200
15625/15625 [==============================] - 1s 46us/sample - loss: 0.0296 - mean_absolute_error: 0.1205 - val_loss: 0.0180 - val_mean_absolute_error: 0.0823
Epoch 139/200
15625/15625 [==============================] - 1s 48us/sample - loss: 0.0299 - mean_absolute_error: 0.1211 - val_loss: 0.0179 - va

15625/15625 [==============================] - 1s 46us/sample - loss: 0.0284 - mean_absolute_error: 0.1163 - val_loss: 0.0181 - val_mean_absolute_error: 0.0831
Epoch 182/200
15625/15625 [==============================] - 1s 46us/sample - loss: 0.0285 - mean_absolute_error: 0.1165 - val_loss: 0.0186 - val_mean_absolute_error: 0.0861
Epoch 183/200
15625/15625 [==============================] - 1s 48us/sample - loss: 0.0288 - mean_absolute_error: 0.1173 - val_loss: 0.0200 - val_mean_absolute_error: 0.0890
Epoch 184/200
15625/15625 [==============================] - 1s 45us/sample - loss: 0.0345 - mean_absolute_error: 0.1330 - val_loss: 0.0206 - val_mean_absolute_error: 0.0935
Epoch 185/200
15625/15625 [==============================] - 1s 47us/sample - loss: 0.0312 - mean_absolute_error: 0.1247 - val_loss: 0.0188 - val_mean_absolute_error: 0.0863
Epoch 186/200
15625/15625 [==============================] - 1s 46us/sample - loss: 0.0304 - mean_absolute_error: 0.1225 - val_loss: 0.0191 - va

In [13]:
def test_mcdropout_regressor(x_test_values, q_test_values, model, mean_scaler, sd_scaler):   
    mc_model = MCDropoutRegressor(model)
    inp = x_test_values  
    
    mean, std = mc_model.predict(inp, num_samples = 10)
    
    q_pred_unnormalised = mean_scaler.inverse_transform(mean)
    
    q_sd_unnromalised = sd_scaler.inverse_transform(std)
    
    global_mae = mean_absolute_error(q_test_values, mean)

    print("Testing MAE: {:.5f}".format(global_mae))

    return q_pred_unnormalised, std
  


In [14]:

mean_1, std_1 = test_mcdropout_regressor(x_test, q_test, model, q_scaler, x_scaler)
q_test_unorm = q_scaler.inverse_transform(q_test)
sd_test_1 = pd.DataFrame(std_1)
print(sd_test_1[0].min())


ValueError: operands could not be broadcast together with shapes (4096,18) (9,) (4096,18) 